# Datasets

- https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset
- https://www.kaggle.com/datasets/tonygordonjr/spotify-dataset-2023?select=spotify-albums_data_2023.csv
- https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks?select=tracks.csv

# Links
- https://forecastegy.com/posts/xgboost-multiclass-classification-python/
- https://github.com/jannine92/spotify_recommendation/blob/main/music_recommender.ipynb
- https://www.kaggle.com/code/nyjoey/spotify-clustering
- https://ausaf-a.github.io/ml-song-recommender/
- https://medium.com/@Marlon_H/spotify-clustering-f41b40003c9a
- https://www.kaggle.com/code/choongqianzheng/song-genre-classification-system
- https://developer.spotify.com/documentation/web-api/reference/get-audio-features
- https://medium.com/@miguelrodrigueznovelo/discover-your-perfect-playlist-10-songs-recommended-by-a-music-recommendation-system-with-python-5fd246d87127
- https://medium.com/@shruti.somankar/building-a-music-recommendation-system-using-spotify-api-and-python-f7418a21fa41
- https://www.kaggle.com/code/merveeyuboglu/music-recommendation-system-cosine-s


# ToDo List:

- Basic stuff✅
  - Load Data✅
  -  Display, Info and Describe data✅
  - Split Datasets into song_metrics and song_info✅
- Data Visualization (Also in Percent if valuable)✅
  - Visualize Correlation Heatmap✅
  - Display Genres as Numbers and Histogram✅
  - Display Genre Dendogram✅
  - Display most frequent artists✅
  - Display most popular artist✅
  - Plot Popularity as histogram✅
  - Plot Average Song metric for Each genre (could also be on a 3D plot)✅
  - Plot Box plots to detect outliers✅
- Features
  - Apply Standard and MinMaxScaler ✅
  - Apply and Visualize PCA and t-SNE / UMAP
  - Use Silhouette Score to see how many clusters are needed (also try fancy plot from Medium)
  - Use KMeans to start
  - Use DBSCAN
  - Use Agglomerative Clustering
  - Use HDBSCAN
  - Use XGBClassifier with Cross Validation
- Feature Extensions
  - Plot Similar Artists
  - Plot Similar Genres
  - (Plot Similar Songs [Only small set of Data here])
- Possible Uses:
  - Put song into spotify api, get song data back, and use that to find similar songs (with possibility to get different artists than the one from the provided song)
  - Put Song in, get similar artist (you could also put multiple songs in, but I dont think that this is worth it)
  - Simulate entering a whole user profile, from which we can take the average song data and get new artists this way (which are not in here)
- Things missing
  - We dont have the release date or listening date, so we cannot use time as a feature. This could create even better recommendations, because we would know what the user currently listens to and weigh it  

# Utils

## Load all Files in Folder

In [4]:
import os
import pandas as pd

def load_and_concatenate_parquet_files(folder_path):
    # List all files in the directory
    files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]
    
    # Sort files for consistent order if needed (optional)
    files.sort()

    # Load and concatenate all the Parquet files
    df_list = []
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_parquet(file_path)
        df_list.append(df)

    # Concatenate all dataframes
    concatenated_df = pd.concat(df_list, ignore_index=True)
    
    return concatenated_df


# Save File in Format for GitHub

In [17]:
import os
import math
import joblib
import pandas as pd

def save_dataframe_as_parquet(df:pd.DataFrame, folder_path="data", folder_name=None, always_overwrite=None, model_object=None):
    if not folder_name:
        folder_name = input("Enter the name of the folder to save the files: ")

    full_path = os.path.join(folder_path, folder_name)

    # Check if the folder already exists
    if os.path.exists(full_path) and always_overwrite is not True:
        if always_overwrite is None:
            overwrite = input(f"The folder '{folder_name}' already exists. Do you want to overwrite it? (yes/no): ")
            always_overwrite = overwrite.lower() != 'yes'
        if not always_overwrite:
            suffix = 1
            new_folder_name = f"{folder_name}_{suffix}"
            while os.path.exists(os.path.join(folder_path, new_folder_name)):
                suffix += 1
                new_folder_name = f"{folder_name}_{suffix}"
            folder_name = new_folder_name
            full_path = os.path.join(folder_path, folder_name)
    
    os.makedirs(full_path, exist_ok=True)

    temp_file = os.path.join(full_path, "temp.parquet")
    df.to_parquet(temp_file)
    file_size = os.path.getsize(temp_file) / (1024 * 1024)  # Size in MB
    os.remove(temp_file)

    if file_size > 50:
        num_splits = math.ceil(file_size / 50)
        row_split = math.ceil(len(df) / num_splits)
    else:
        num_splits = 1
        row_split = len(df)
    
    for i in range(num_splits):
        start_row = i * row_split
        end_row = min((i + 1) * row_split, len(df))
        split_df = df.iloc[start_row:end_row]
        split_file_name = os.path.join(full_path, f"{folder_name}_part_{i + 1}.parquet")
        split_df.to_parquet(split_file_name)
    
    if model_object:
        for key, value in model_object.items():
            joblib.dump(value, f'{full_path}/{key}_model.pkl')

    print(f"Dataframe saved in {num_splits} files under the folder: {full_path}")
    
    return full_path

# Example usage:
# save_dataframe_as_parquet(df=kmeans_cluster, folder_path="data", folder_name="kmeans_clustered_subset", always_overwrite=False)


# Load and View Dataset

In [6]:
import pandas as pd

original_data = load_and_concatenate_parquet_files("data/spotify_data")

original_data["year"] = pd.to_datetime(original_data["year"], format='%Y')
original_data = original_data.dropna(subset=["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature", "popularity", "track_id", "track_name", "artist_name", "year"])
original_data = original_data.drop_duplicates(subset=["track_name", "artist_name", "danceability", "energy", "key", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"])
original_data = original_data.reset_index(drop=True)
original_data = original_data.drop(columns=["Unnamed: 0"])
display(original_data.head())
display(original_data.describe())
print(original_data.info())

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68.0,2012-01-01,acoustic,0.483,0.303,4.0,-10.058,1.0,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166.0,3.0
1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50.0,2012-01-01,acoustic,0.572,0.454,3.0,-10.286,1.0,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387.0,4.0
2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57.0,2012-01-01,acoustic,0.409,0.234,3.0,-13.711,1.0,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960.0,4.0
3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58.0,2012-01-01,acoustic,0.392,0.251,10.0,-9.845,1.0,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293.0,4.0
4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54.0,2012-01-01,acoustic,0.430,0.791,6.0,-5.419,0.0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320.0,4.0


,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,2.013313e+06,2013313,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06,2.013313e+06
mean,1.848877e+01,2005-10-05 01:00:34.158026624,5.491744e-01,5.904303e-01,5.256363e+00,-9.799186e+00,6.422682e-01,1.010894e-01,3.714914e-01,2.211444e-01,2.162194e-01,4.809441e-01,1.200666e+02,2.385117e+05,3.878141e+00
min,0.000000e+00,1886-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,-6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.073000e+03,0.000000e+00
25%,3.000000e+00,2002-01-01 00:00:00,4.280000e-01,3.870000e-01,2.000000e+00,-1.217600e+01,0.000000e+00,3.600000e-02,2.060000e-02,0.000000e+00,9.790000e-02,2.540000e-01,9.700300e+01,1.751250e+05,4.000000e+00
50%,1.500000e+01,2011-01-01 00:00:00,5.650000e-01,6.220000e-01,5.000000e+00,-8.334000e+00,1.000000e+00,4.870000e-02,2.490000e-01,4.380000e-04,1.330000e-01,4.740000e-01,1.200100e+02,2.185470e+05,4.000000e+00
75%,3.000000e+01,2018-01-01 00:00:00,6.860000e-01,8.250000e-01,8.000000e+00,-5.815000e+00,1.000000e+00,8.800000e-02,7.210000e-01,4.240000e-01,2.790000e-01,7.020000e-01,1.382540e+02,2.739870e+05,4.000000e+00
max,1.000000e+02,2023-01-01 00:00:00,9.990000e-01,1.000000e+00,1.100000e+01,6.172000e+00,1.000000e+00,9.710000e-01,9.960000e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.499930e+02,6.000495e+06,5.000000e+00
std,1.674866e+01,NaN,1.819063e-01,2.708000e-01,3.549231e+00,5.787686e+00,4.793327e-01,1.522333e-01,3.599918e-01,3.519784e-01,1.926347e-01,2.695143e-01,2.996391e+01,1.405136e+05,4.832763e-01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013313 entries, 0 to 2013312
Data columns (total 19 columns):
 #   Column            Dtype         
---  ------            -----         
 0   artist_name       object        
 1   track_name        object        
 2   track_id          object        
 3   popularity        float64       
 4   year              datetime64[ns]
 5   genre             object        
 6   danceability      float64       
 7   energy            float64       
 8   key               float64       
 9   loudness          float64       
 10  mode              float64       
 11  speechiness       float64       
 12  acousticness      float64       
 13  instrumentalness  float64       
 14  liveness          float64       
 15  valence           float64       
 16  tempo             float64       
 17  duration_ms       float64       
 18  time_signature    float64       
dtypes: datetime64[ns](1), float64(14), object(4)
memory usage: 291.8+ MB
None


# Recommendation Engine code

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import hdbscan
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

def reduce_data(data, dimensions):
    numeric_columns = data[metric_columns]
    pca_standardized = PCA(n_components=dimensions)
    pca_standardized_result = pca_standardized.fit_transform(numeric_columns)
    return pca_standardized_result

def normalized_data(data:pd.DataFrame):
    min_max_scaler = MinMaxScaler()
    numeric_columns = data[metric_columns]
    other_columns = data.drop(columns=metric_columns).reset_index(drop=True)
    normalized_data = min_max_scaler.fit_transform(numeric_columns)
    normalized_df = pd.DataFrame(normalized_data, columns=numeric_columns.columns)
    normalized_data = pd.merge(normalized_df, other_columns, left_index=True, right_index=True)
    return normalized_df

def run_kmeans_and_hdbscan(original_data:pd.DataFrame, cluster_data:pd.DataFrame, kmeans_clusters=5, hdbscan_min_cluster_size=10):
    data_for_clustering = cluster_data.copy()
    kmeans = KMeans(n_clusters=kmeans_clusters, random_state=42)
    kmeans_labels = kmeans.fit_predict(data_for_clustering)

    data_for_clustering["kmeans_cluster"] = kmeans_labels
    original_data["kmeans_cluster"] = kmeans_labels
    print(f"KMeans_cluster distro: {data_for_clustering['kmeans_cluster'].value_counts()}")
    hdbscan_cluster_df = []
    hdbscan_models = {}
    for cluster, data in data_for_clustering.groupby("kmeans_cluster"):
        hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=hdbscan_min_cluster_size)
        hdbscan_cluster_labels = hdbscan_clusterer.fit_predict(data)
        current_original_data = original_data[original_data["kmeans_cluster"] == cluster].reset_index(drop=True).copy()
        current_original_data["hdbscan_cluster"] = hdbscan_cluster_labels
        hdbscan_cluster_df.append(current_original_data)
        hdbscan_models[cluster] = hdbscan_clusterer
        print(f"HDBSCAN_cluster distro for kmeans {cluster}: {current_original_data['hdbscan_cluster'].value_counts()}")

    all_cluster_results = pd.concat(hdbscan_cluster_df).reset_index(drop=True)
    
    return all_cluster_results, kmeans, hdbscan_models


def cluster_data(original_data:pd.DataFrame, subset_fraction=None, kmeans=True, use_hdbscan=True, kmeans_clusters=5, hdbscan_min_cluster_size=10, standardize_data=True, pca_dimensions=None, columns_to_use=None):
    
    if subset_fraction:
        original_data = original_data.sample(frac=subset_fraction)
    
    data_for_clustering = original_data.copy().reset_index(drop=True)
    
    if standardize_data:
        data_for_clustering = normalized_data(data_for_clustering)

    if pca_dimensions:
        data_for_clustering = reduce_data(data_for_clustering, pca_dimensions)
    
    if pca_dimensions is None:
        metric_columns = columns_to_use if columns_to_use else ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"]
        data_for_clustering = data_for_clustering[metric_columns]

    if kmeans and use_hdbscan:
        result_df, kmeans, hdbscan_models = run_kmeans_and_hdbscan(original_data, data_for_clustering, kmeans_clusters, hdbscan_min_cluster_size)
        folder_path = save_dataframe_as_parquet(result_df, folder_path="data", folder_name="kmeans_hdbscan_clustered_subset", always_overwrite=False, model_object={"kmeans": kmeans})
        for key, value in hdbscan_models.items():
            joblib.dump(value, f'{folder_path}/hdbscan_model_{key}.pkl')
        result = result_df.copy()
    
    elif kmeans:
        kmeans = KMeans(n_clusters=kmeans_clusters, random_state=42)
        kmeans_labels = kmeans.fit_predict(data_for_clustering)
        original_data["kmeans_cluster"] = kmeans_labels
        save_dataframe_as_parquet(original_data, folder_path="data", folder_name="kmeans_clustered_subset", always_overwrite=False, model_object={"kmeans": kmeans})
        print(f"KMeans_cluster distro: {original_data['kmeans_cluster'].value_counts()}")
        result = original_data.copy()
        
    elif use_hdbscan:
        hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=hdbscan_min_cluster_size)
        hdbscan_labels = hdbscan_clusterer.fit_predict(data_for_clustering)
        original_data["hdbscan_cluster"] = hdbscan_labels
        save_dataframe_as_parquet(original_data, folder_path="data", folder_name="hdbscan_clustered_subset", always_overwrite=False, model_object={"hdbscan": hdbscan_clusterer})
        print(f"HDBSCAN_cluster distro: {original_data['hdbscan_cluster'].value_counts()}")
        result = original_data.copy()

    return result

clustered_data = cluster_data(original_data, subset_fraction=0.01, kmeans=True, use_hdbscan=False, kmeans_clusters=5, hdbscan_min_cluster_size=10, standardize_data=True, pca_dimensions=None, columns_to_use=None)
display(clustered_data)

Dataframe saved in 1 files under the folder: data/kmeans_clustered_subset_1
KMeans_cluster distro: kmeans_cluster
2    5690
1    4588
0    3818
3    3712
4    2325
Name: count, dtype: int64


,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,kmeans_cluster
1422250,Nikos Oikonomopoulos,Etsi Nomizis (Ke Mi Girisis),2HSZYfpV0Ekn1SLoEM7Hv7,24.0,2009-01-01,None,0.502,0.68000,11.0,-5.776,0.0,0.0458,0.38000,0.000000,0.3980,0.4420,92.053,245560.0,4.0,1
1981539,Neil Diamond,Lost In Hollywood,01RlA9MxNRgmpY07diyu4p,23.0,1986-01-01,None,0.754,0.55100,7.0,-10.949,1.0,0.0294,0.07590,0.000000,0.0505,0.7470,127.864,261600.0,4.0,2
713715,The Future Sound Of London,Papua New Guinea (Mellow Magic Maze Mix),4adx3PhkJAog6RQPhUOhEL,4.0,2002-01-01,breakbeat,0.513,0.88800,3.0,-8.796,1.0,0.0373,0.02410,0.400000,0.3450,0.4840,124.985,330791.0,4.0,3
992052,Paul Baloche,All Praise and Honor - Live,6Bxkrsitofy3w2mxxszYWH,8.0,2008-01-01,french,0.391,0.48200,9.0,-8.308,1.0,0.0316,0.52400,0.000000,0.3590,0.1880,142.795,443067.0,4.0,3
305086,Dog Music,The Best Dog Music,13fECrGugy30hYjI2h9zim,21.0,2017-01-01,sleep,0.130,0.00102,3.0,-37.190,1.0,0.0469,0.96500,0.737000,0.1000,0.0929,79.356,118387.0,5.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431807,Eddie Griffin,"""We Need to Talk""",6XMcxsYbrpAWUexgn3uTBw,11.0,2020-01-01,comedy,0.633,0.66500,1.0,-11.775,1.0,0.9260,0.80500,0.000000,0.8380,0.6240,103.190,271018.0,3.0,0
1797597,Memphis Reigns,Red Vines,6eXb14CbsaxiSCBUEbZm6T,0.0,2023-01-01,None,0.426,0.69800,9.0,-9.564,1.0,0.3680,0.36300,0.000002,0.2930,0.2920,173.143,223256.0,4.0,3
1701413,Danstar Lord,Go Away,2wDHQJ4tpZR158tCaPP7Ex,0.0,2023-01-01,None,0.876,0.68700,4.0,-5.589,1.0,0.0507,0.02360,0.000000,0.0671,0.7190,139.982,179929.0,4.0,2
1222252,Black Sheep,The Choice Is Yours (Revisited),4k9EkhkFZY8Bk41Qi0Ob7P,59.0,1991-01-01,None,0.867,0.67900,10.0,-12.028,0.0,0.2070,0.03360,0.000002,0.0742,0.6800,101.293,243200.0,4.0,1


## Actual Functionality

#### Config

In [ ]:
from rapidfuzz import process, utils
from sklearn.neighbors import NearestNeighbors
import hdbscan
import joblib
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import os
from dotenv import load_dotenv
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

class MusicRecommendation:
    def __init__(self, original_data: pd.DataFrame, weight_year: float, weight_popularity: float):

        load_dotenv()

        # Initialize the Spotify client with authentication
        auth_manager = SpotifyOAuth(client_id=os.getenv('SPOTIFY_CLIENT_ID'), client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'), redirect_uri=os.getenv('SPOTIFY_REDIRECT_URI'), scope="user-library-read")
        self.sp = spotipy.Spotify(auth_manager=auth_manager)
        self.original_data = original_data
        self.weight_year = weight_year
        self.weight_popularity = weight_popularity
        self.clustering_method = "kmeans"
        self.hdbscan_model = joblib.load("hdbscan_model.pkl")
        self.kmeans_model = joblib.load("kmeans_model.pkl")
        self.clustered_data = pd.read_parquet("clustered_subset.parquet") if self.clustering_method == "hdbscan" else load_and_concatenate_parquet_files("data/kmean_clustered_subset")
        self.metrics = [
            'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
            'time_signature'
        ]
    
    def get_track_features(self, song_name: str, artist_name: str) -> pd.DataFrame:
        results = self.sp.search(q=f"track:{song_name} artist:{artist_name}", type="track", limit=1)
        
        if not results['tracks']['items']:
            print("Track not found")
            return pd.DataFrame()  # Return an empty DataFrame if the track is not found
        
        track = results['tracks']['items'][0]
        track_id = track['id']
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        popularity = track['popularity']
        release_date = track['album']['release_date']
        year = int(release_date.split('-')[0])
        duration_ms = track['duration_ms']
        audio_features = self.sp.audio_features(track_id)[0]
        
        data = {
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'key': audio_features['key'],
            'loudness': audio_features['loudness'],
            'mode': audio_features['mode'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'valence': audio_features['valence'],
            'tempo': audio_features['tempo'],
            'time_signature': audio_features['time_signature'],
            'artist_name': artist_name,
            'track_name': track_name,
            'track_id': track_id,
            'popularity': popularity,
            'year': year,
            'duration_ms': duration_ms
        }
        return pd.DataFrame([data])

    def normalized_data(self, data: pd.DataFrame) -> pd.DataFrame:
        min_max_scaler = MinMaxScaler()
        numeric_columns = data[self.metrics]
        other_columns = data.drop(columns=self.metrics).reset_index(drop=True)
        normalized_data = min_max_scaler.fit_transform(numeric_columns)
        return pd.merge(pd.DataFrame(normalized_data, columns=self.metrics), other_columns, left_index=True, right_index=True)
    
    def reduce_data(self, data: pd.DataFrame, dimensions: int) -> np.ndarray:
        numeric_columns = data[self.metrics]
        pca = PCA(n_components=dimensions)
        return pca.fit_transform(numeric_columns)

    def get_closest_match(self, user_input: str, df: pd.DataFrame, column: str, threshold: int = 92) -> str:
        processed_user_input = utils.default_process(user_input)
        strings_column = df[column].dropna()
        processed_strings = [utils.default_process(string) for string in strings_column]
        match = process.extractOne(processed_user_input, processed_strings, processor=None, score_cutoff=threshold)
        return strings_column.iloc[match[2]] if match else None

    def song_finder(self, song_name: str, artist_name: str, data) -> pd.DataFrame:
        song = data[(data["track_name"] == song_name) & (data["artist_name"] == artist_name)]
        return song if not song.empty else None

    def preprocess_song(self, song: pd.DataFrame, normalization: str, reduction: str) -> pd.DataFrame:
        if normalization == "standardized":
            song = self.standardized_data(song)
        elif normalization == "normalized":
            song = self.normalized_data(song)
        if reduction == "pca":
            song = self.reduce_data(song, 2)
        return song

    def get_song_cluster(self, song: pd.DataFrame) -> int:
        if self.clustering_method == "hdbscan":
            new_data_point = song[self.metrics].values.reshape(1, -1)
            predicted_cluster, _ = hdbscan.approximate_predict(self.hdbscan_model, new_data_point)
            cluster = predicted_cluster[0]
            
        elif self.clustering_method == "kmeans":
            cluster = self.kmeans_model.predict(song[self.metrics])[0]
        return cluster

    def find_nearest_neighbors(self, song: pd.DataFrame, data: pd.DataFrame, number_of_songs: int) -> tuple:
        knn_model = NearestNeighbors(n_neighbors=100)
        cluster_data = data[self.metrics]
        knn_model.fit(cluster_data)
        distances, indices = knn_model.kneighbors(song[self.metrics], n_neighbors=100)
        neighbors_df = data.iloc[indices[0]]
        return neighbors_df, distances[0]

    def get_weighted_scores(self, neighbors_df: pd.DataFrame, neighbor_distances: np.ndarray) -> pd.DataFrame:
        # Ensure we are working with a copy of the DataFrame to avoid SettingWithCopyWarning
        neighbors_df = neighbors_df.copy()
        
        
        
        # Convert 'year' column to numeric
        neighbors_df['year'] = pd.to_datetime(neighbors_df['year'], format='%Y')
        neighbors_df['year_numeric'] = neighbors_df['year'].dt.year
        
        # Normalize 'year_numeric' and 'popularity' columns
        scaler = MinMaxScaler()
        neighbors_df[['year_normalized', 'popularity_normalized']] = scaler.fit_transform(
            neighbors_df[['year_numeric', 'popularity']]
        )
        
        year_normalized = neighbors_df['year_normalized'].values
        popularity_normalized = neighbors_df['popularity_normalized'].values
        
        # Define weights for year and popularity
        year_weight = self.weight_year
        popularity_weight = self.weight_popularity

        # Compute base scores (inverse distance, to ensure higher similarity has a higher base score)
        base_scores = 1 / (neighbor_distances + 1e-8)  # Avoid division by zero

        # Compute boosting scores
        boosting_scores = year_normalized * year_weight + popularity_normalized * popularity_weight

        # Final scores: add boosting scores to base scores
        final_scores = base_scores + boosting_scores

        # Rank neighbors based on weighted scores
        ranked_indices = np.argsort(final_scores)[::-1]  # Sort in descending order
        return neighbors_df.iloc[ranked_indices]

    def print_preview_urls(self, song_df: pd.DataFrame) -> None:
        for _, row in song_df.iterrows():
            track_id = row['track_id']
            track = self.sp.track(track_id)
            preview_url = track.get('preview_url')
            if preview_url:
                print(f"Track: {row['track_name']} by {row['artist_name']}")
                print(f"Preview URL: {preview_url}")
            else:
                print(f"Track: {row['track_name']} by {row['artist_name']}")
                print("Preview URL not available.")
    
    def choose_correct_song(self, song: pd.DataFrame) -> pd.DataFrame:
        if len(song) > 1:
            chosen_index = int(input("Enter the index of the song you want to use: "))
            if chosen_index >= len(song):
                print("Invalid index, using 1st song")
                chosen_index = 0
            song = song.iloc[chosen_index].copy()
        return song
    
    def remove_song_if_present(self, song: pd.DataFrame, data: pd.DataFrame) -> pd.DataFrame:
        matched_rows = data[(data["track_name"] == song["track_name"].values[0]) & (data["artist_name"] == song["artist_name"].values[0])]
        if not matched_rows.empty:
            print(f"Removing {len(matched_rows)} rows from possible recommendations")
            if len(matched_rows) > 1:
                print("Rows to be removed:")
                display(matched_rows.head(5))
            indexes_to_remove = matched_rows.index
            return data.drop(indexes_to_remove)
        else:
            return data
    
    
    def find_closest_songs(self, song_name: str = "", artist_name: str = "", same_artist: bool = "", number_of_songs: int = "", weighted_system: bool = "") -> pd.DataFrame:
        if artist_name == "":
            artist_name = input("Enter the artist name: ")
        if song_name == "":
            song_name = input("Enter the song name: ")
        if same_artist == "":
            same_artist = input("Filter by same artist? (yes/no): ").strip().lower() == 'yes'
        if number_of_songs == "":
            number_of_songs = int(input("Enter the number of songs to return: "))
        if weighted_system == "":
            weighted_system = input("Use weighted system? (yes/no): ").strip().lower() == 'yes'

        song = self.get_track_features(song_name, artist_name)
        
        if song is None:
            print("No match found")
            return None
        
        print("Song found")
        real_artist_name = song["artist_name"].values[0]
        real_song_name = song["track_name"].values[0]
        print(f"Using {real_song_name} by {real_artist_name}")
        display(song.reset_index(drop=True))
        self.print_preview_urls(song)
        
        song = self.choose_correct_song(song)
        
        clustered_data = self.remove_song_if_present(song, self.clustered_data)
        
        all_songs = pd.concat([clustered_data, song], ignore_index=True) 
        
        all_song_standardized = self.normalized_data(all_songs)
        song_standardized = self.song_finder(real_song_name, real_artist_name, all_song_standardized)
        
        predicted_cluster = self.get_song_cluster(song_standardized)
        
        print(f"\nPredicted cluster: {predicted_cluster}\n")
        
        sample_data = all_song_standardized[all_song_standardized["cluster"] == predicted_cluster]
        if not same_artist:
            sample_data = sample_data[sample_data["artist_name"] != artist_name]
         
        neighbors_df, neighbor_distances = self.find_nearest_neighbors(song_standardized, sample_data, number_of_songs)
        
        if weighted_system:
            neighbors_df = self.get_weighted_scores(neighbors_df, neighbor_distances)
        
        closest_songs = self.original_data.loc[neighbors_df.head(number_of_songs).index]
        
        closest_songs = closest_songs[song.columns]
        
        self.print_preview_urls(closest_songs)
        
        return closest_songs

# Example usage:
if __name__ == "__main__":
    # Initialize with default paths and parameters
   
    recommender = MusicRecommendation(original_data = original_data, weight_year=0.6, weight_popularity=0.4)

    # Find closest songs with

recommender.find_closest_songs(artist_name="Skrillex", song_name="bangarang", same_artist=True, number_of_songs=5, weighted_system=False)


# Here the modelling and transformation starts

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Select the numeric columns
numeric_columns = feature_df.drop(columns=["track_id", "genre"])

standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

# Standardize the numeric columns
standardized_data = standard_scaler.fit_transform(numeric_columns)
standardized_df = pd.DataFrame(standardized_data, columns=numeric_columns.columns)
standardized_df['genre'] = feature_df['genre']
standardized_df['track_id'] = feature_df['track_id']

# Normalize the numeric columns
normalized_data = min_max_scaler.fit_transform(numeric_columns)
normalized_df = pd.DataFrame(normalized_data, columns=numeric_columns.columns)
normalized_df['genre'] = feature_df['genre']
normalized_df['track_id'] = feature_df['track_id']

# Display the standardized and normalized dataframes
display(standardized_df.describe())
display(normalized_df.describe())


In [ ]:
display(standardized_df.isna().sum())

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import plotly.express as px

# Perform PCA on standardized_data
pca_standardized = PCA(n_components=2)
pca_standardized_result = pca_standardized.fit_transform(standardized_data)
print(1)

# Perform PCA on normalized_data
pca_normalized = PCA(n_components=2)
pca_normalized_result = pca_normalized.fit_transform(normalized_data)
print(2)

# # Perform t-SNE on standardized_data
# tsne_standardized = TSNE(n_components=2)
# tsne_standardized_result = tsne_standardized.fit_transform(standardized_data)
# print(3)

# # Perform t-SNE on normalized_data
# tsne_normalized = TSNE(n_components=2)
# tsne_normalized_result = tsne_normalized.fit_transform(normalized_data)
# print(4)

# # Create the subplot with 4 plots
# fig = px.subplots(
#     rows=2, cols=2,
#     subplot_titles=("PCA - Standardized Data", "PCA - Normalized Data", "t-SNE - Standardized Data", "t-SNE - Normalized Data"),
#     shared_xaxes=True, shared_yaxes=True,
#     vertical_spacing=0.1, horizontal_spacing=0.1
# )

# # Add PCA - Standardized Data plot
# fig.add_trace(
#     px.scatter(x=pca_standardized_result[:, 0], y=pca_standardized_result[:, 1], color=standardized_df['track_genre']).data[0],
#     row=1, col=1
# )

# # Add PCA - Normalized Data plot
# fig.add_trace(
#     px.scatter(x=pca_normalized_result[:, 0], y=pca_normalized_result[:, 1], color=normalized_df['track_genre']).data[0],
#     row=1, col=2
# )

# # Add t-SNE - Standardized Data plot
# fig.add_trace(
#     px.scatter(x=tsne_standardized_result[:, 0], y=tsne_standardized_result[:, 1], color=standardized_df['track_genre']).data[0],
#     row=2, col=1
# )

# # Add t-SNE - Normalized Data plot
# fig.add_trace(
#     px.scatter(x=tsne_normalized_result[:, 0], y=tsne_normalized_result[:, 1], color=normalized_df['track_genre']).data[0],
#     row=2, col=2
# )

# # Update layout
# fig.update_layout(
#     height=800,
#     showlegend=False
# )

# # Show the subplot
# fig.show()

px.scatter(x=pca_standardized_result[:, 0], y=pca_standardized_result[:, 1], color=standardized_df['genre']).show()

In [ ]:
# sns.pairplot(original_data, hue='track_genre', diag_kind='kde')

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.manifold import TSNE

# import plotly.express as px

# dataframe = standardized_df.copy()
# # Assuming 'data' is your dataframe and track_genre is a column in the dataframe

# # Create a subset of the data
# subset_data = dataframe.sample(n=1000, random_state=42)

# # Prepare the data: Separate features and labels
# features = subset_data.drop(columns=['track_genre', "track_id"])  # Drop the track_genre column
# labels = subset_data['track_genre']  # Save the track_genre column separately

# # Apply t-SNE
# tsne = TSNE(n_components=2, random_state=42)
# tsne_results = tsne.fit_transform(features)

# # Create a DataFrame for the t-SNE results
# tsne_df = pd.DataFrame(tsne_results, columns=['tsne_1', 'tsne_2'])
# tsne_df['track_genre'] = labels.values

# # Plot the results using Plotly Express
# fig = px.scatter(tsne_df, x='tsne_1', y='tsne_2', color='track_genre', title='t-SNE of Track Features by Genre')
# fig.show()


In [ ]:
# import umap

# reducer = umap.UMAP(n_components=2, random_state=42)

# # Apply UMAP
# umap_results = reducer.fit_transform(subset_data.drop(columns=['track_genre', "track_id"]))

# px.scatter(x=umap_results[:, 0], y=umap_results[:, 1], color=subset_data['track_genre']).show()

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from datetime import datetime
# from sklearn.metrics.pairwise import cosine_similarity

# # a function to get content-based recommendations based on music features
# def content_based_recommendations(input_song_name, num_recommendations=5):
#     if input_song_name not in music_df['Track Name'].values:
#         print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
#         return

#     # Get the index of the input song in the music DataFrame
#     input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

#     # Calculate the similarity scores based on music features (cosine similarity)
#     similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

#     # Get the indices of the most similar songs
#     similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

#     # Get the names of the most similar songs based on content-based filtering
#     content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

#     return content_based_recommendations

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Loop through a range of cluster numbers to calculate silhouette scores
silhouette_scores = []
cluster_range = range(2, 26)
data_sample = standardized_data[np.random.choice(standardized_data.shape[0], 20000, replace=False)]
for k in cluster_range:
    print(f"Calculating silhouette score for k = {k}")
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(data_sample)
    silhouette_avg = silhouette_score(data_sample, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {k}, the average silhouette score is {silhouette_avg:.4f}")

# Optionally, you can plot the silhouette scores
import matplotlib.pyplot as plt

plt.plot(cluster_range, silhouette_scores, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores for k-means clustering')
plt.show()


In [ ]:
import numpy as np
from sklearn.cluster import HDBSCAN

data_sample = standardized_data[np.random.choice(standardized_data.shape[0], 200000, replace=False)]

# Fit the HDBSCAN model
hdbscan_model = HDBSCAN(min_cluster_size=100)
hdbscan_model.fit(data_sample)

# Get the labels assigned to each data point
cluster_labels = hdbscan_model.labels_

# Example: Print out the first 10 cluster labels
print("First 10 cluster labels:", cluster_labels[:10])

# Print out the number of clusters found (excluding noise)
print(f"Number of clusters found: {len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)}")


In [ ]:
def song_finder(song_name, artist_name):
    song = clustered_data[(clustered_data["track_name"] == song_name) & (clustered_data["artist_name"] == artist_name)]
    return song

song = song_finder("Shape of You", "Ed Sheeran")

standardized_data[song.index]

for song in clustered_data[['track_name', 'artist_name']].itertuples():
    print(song[0])
    print(standardized_data[song[0]])

In [ ]:
from scipy.spatial import distance

def song_finder(song_name, artist_name):
    song = clustered_data[(clustered_data["track_name"] == song_name) & (clustered_data["artist_name"] == artist_name)]
    return song

def find_closest_songs(song_name, artist_name, song_number=5):
    all_distances = []
    
    chosen_song = song_finder(song_name, artist_name)
    index = chosen_song.index
    print(index)
    print(standardized_data[index][0])
    for song in clustered_data[['track_name', 'artist_name']].itertuples():

        current_distance = distance.cosine(standardized_data[song[0]],standardized_data[chosen_song.index][0])
        all_distances.append((song.track_name, song.artist_name, current_distance))
    all_distances.sort(key=lambda x: x[2], reverse=False)
    return all_distances[1:song_number+1]

find_closest_songs("Shape of You", "Skrillex")